In [165]:
import rasterio as rio
import matplotlib.pyplot as plt 
import numpy as np
import numpy.matlib
from scipy import interpolate
import statsmodels.api as sm
import scipy.stats as st
import os, sys, pickle, gzip
import datetime
import geopy.distance
import xarray as xr
import xesmf as xe
import cartopy.crs as ccrs
import glob


In [166]:
run ../util/setupConsole

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
# dirAgData = '/home/edcoffel/drive/MAX-Filer/Research/Climate-01/Personal-F20/edcoffel-F20/data/projects/ag-land-climate'
# dirEra5 = '/home/edcoffel/drive/MAX-Filer/Research/Climate-02/Data-02-edcoffel-F20/ERA5'

dirAgData = '/dartfs-hpc/rc/lab/C/CMIG/ecoffel/data/projects/ag-land-climate'
dirEra5 = '/dartfs-hpc/rc/lab/C/CMIG/ERA5'

years = [1979, 2019]

In [168]:
sacksMaizeNc = xr.open_dataset('%s/sacks/Maize.crop.calendar.fill.nc'%dirAgData)
sacksStart = sacksMaizeNc['plant'].values
sacksStart = np.roll(sacksStart, -int(sacksStart.shape[1]/2), axis=1)
sacksStart[sacksStart < 0] = np.nan
sacksEnd = sacksMaizeNc['harvest'].values
sacksEnd = np.roll(sacksEnd, -int(sacksEnd.shape[1]/2), axis=1)
sacksEnd[sacksEnd < 0] = np.nan

sacksLat = np.linspace(90, -90, 360)
sacksLon = np.linspace(0, 360, 720)

In [169]:
# calc era5 quantiles
ds_tasmax = xr.open_mfdataset('%s/monthly/tasmax_*.nc'%dirEra5)
ds_tasmax.load()
ds_tasmax['mx2t'] -= 273.15

ds_tp = xr.open_mfdataset('%s/monthly/tp_*.nc'%dirEra5)
ds_tp.load()
# ds_quantile = ds.chunk({'latitude':500, 'longitude':500, 'time': -1}).quantile(q=[.01, .05, .1, .9, .95, .99], dim='time')
# ds_quantile.tw.to_netcdf('era5_tw_mean_quantiles.nc')

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 480)
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * time       (time) datetime64[ns] 1979-01-31 1979-02-28 ... 2018-12-31
Data variables:
    tp         (time, latitude, longitude) float64 8.971 8.971 ... 10.47 10.47

In [170]:
lat = ds_tasmax.latitude.values
lon = ds_tasmax.longitude.values

# regrid sacks data
regridMesh = xr.Dataset({'lat': (['lat'], lat),
                         'lon': (['lon'], lon),})

regridder_start = xe.Regridder(xr.DataArray(data=sacksStart, dims=['lat', 'lon'], coords={'lat':sacksLat, 'lon':sacksLon}), regridMesh, 'bilinear')
regridder_end = xe.Regridder(xr.DataArray(data=sacksEnd, dims=['lat', 'lon'], coords={'lat':sacksLat, 'lon':sacksLon}), regridMesh, 'bilinear')

sacksStart_regrid = regridder_start(sacksStart)
sacksEnd_regrid = regridder_end(sacksEnd)

In [171]:
def is_growing_season(month, sacks_start_month, sacks_end_month):
    if sacks_start_month < sacks_end_month:
        return (month >= sacks_start_month) & (month <= sacks_end_month)
    else:
        return (month >= sacks_start_month) | (month <= sacks_end_month)

In [172]:
with open('maize_hw_2005_regrid.dat', 'rb') as f:
    maize_ha_regrid = pickle.load(f)

In [173]:
nnLen = len(np.where( (~np.isnan(np.reshape(sacksStart_regrid, [sacksStart_regrid.size, 1]))) & (np.reshape(maize_ha_regrid, [maize_ha_regrid.size, 1]) > 0))[0])

In [ ]:
%%time
n = 0

growing_season_tasmax_quantiles = np.full([len(lat), len(lon), 4], np.nan)
growing_season_tp_quantiles = np.full([len(lat), len(lon), 4], np.nan)

for xlat in range(len(lat)):

    for ylon in range(len(lon)):

        if ~np.isnan(sacksStart_regrid[xlat, ylon]) and ~np.isnan(sacksEnd_regrid[xlat, ylon]) and maize_ha_regrid[xlat, ylon] > 0:

            if n % 1000 == 0:
                print('%.1f %% complete'%(n/(nnLen)*100))

            sacks_start_month = datetime.datetime.strptime('2020%d'%int(sacksStart_regrid[xlat,ylon]), '%Y%j').date().month
            sacks_end_month = datetime.datetime.strptime('2020%d'%int(sacksEnd_regrid[xlat,ylon]), '%Y%j').date().month
            
            # select growing seasons for all years
            ds_tasmax_growing = ds_tasmax.mx2t[:,xlat,ylon].sel(time=is_growing_season(ds_tasmax['time.month'], sacks_start_month, sacks_end_month))
            ds_tp_growing = ds_tp.tp[:,xlat,ylon].sel(time=is_growing_season(ds_tasmax['time.month'], sacks_start_month, sacks_end_month))

            # resample to group growing seasons (1 year frequency, offset starting at 1 month before start of growing season)
            ds_tasmax_growing_1y = ds_tasmax_growing.resample(time='1Y', loffset='%dM'%(sacks_start_month-1)).mean()
            ds_tp_growing_1y = ds_tp_growing.resample(time='1Y', loffset='%dM'%(sacks_start_month-1)).mean()

            # calculate quantiles across all growing seasons
            tasmax_q = ds_tasmax_growing_1y.chunk({'time': -1}).quantile(q=[.75, .9, .95, .99], dim='time', skipna=True)
            tp_q = ds_tp_growing_1y.chunk({'time': -1}).quantile(q=[.75, .9, .95, .99], dim='time', skipna=True)

            growing_season_tasmax_quantiles[xlat, ylon, :] = tasmax_q.values
            growing_season_tp_quantiles[xlat, ylon, :] = tp_q.values

            n += 1
                
#                 sys.exit()

0.0 % complete
1.1 % complete
2.2 % complete
3.3 % complete
4.4 % complete
5.5 % complete
6.6 % complete
7.7 % complete
8.8 % complete
9.9 % complete
11.0 % complete
12.1 % complete
13.2 % complete
14.3 % complete
15.4 % complete
16.5 % complete
17.6 % complete
18.7 % complete
19.8 % complete
20.9 % complete
22.0 % complete
23.1 % complete
24.2 % complete
25.4 % complete
26.5 % complete
27.6 % complete
28.7 % complete
29.8 % complete
30.9 % complete
32.0 % complete
33.1 % complete
34.2 % complete
35.3 % complete
36.4 % complete
37.5 % complete
38.6 % complete
39.7 % complete
40.8 % complete
41.9 % complete
43.0 % complete
44.1 % complete
45.2 % complete
46.3 % complete
47.4 % complete
48.5 % complete
49.6 % complete
50.7 % complete
51.8 % complete
